In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import unicodedata
import numpy as np
import itertools

url = 'https://www.transfermarkt.com/premier-league/transfers/wettbewerb/GB1/plus/?saison_id=2018&s_w=&leihe=0&leihe=1&intern=0&intern=1'

response = requests.get(url, headers={'User-Agent': 'Custom5'})
print(response.status_code)
financial_data = response.text
soup = BeautifulSoup(financial_data, 'html.parser')

200


In [2]:
# Install
# !pip install pyjsparser
# !pip install js2xml

# Base
import pandas as pd
import numpy as np

# Visualization
import plotly.express as ex

# Web Scraping
import requests
from bs4 import BeautifulSoup
# !pip install pyjsparser
# from pyjsparser import parse
import pyjsparser

# Time Sleep
import sys
import time
from datetime import datetime
from termcolor import colored

# GC
import gc

# Itertools
import itertools

# Grafikten Data Çekmek için
import re
import js2xml
from itertools import repeat    
from pprint import pprint as pp

# String Manipulation
import re 

# Configurations
import warnings
warnings.filterwarnings("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)
pd.set_option('display.max_columns', None)

# Helpers
# -----------------------------------------------------------------------------
# To learn process time
def get_time(cond):
    if cond == "start":
        p = "This process started at "
    elif cond == "end":
        p = "The process completed at "
    print("") 
    print(colored(p + str(datetime.now().strftime("%H:%M:%S")), "green", "on_white", attrs=["bold",'reverse', 'blink']))
    
    
def Filter(string, substr): 
        return [str for str in string if
                any(sub in str for sub in substr)] 
    
def NOTFilter(string, substr): 
    return [str for str in string if
            any(sub not in str for sub in substr)] 

In [7]:
# Leagues & Seasons
leagues = [
    "https://www.transfermarkt.com/premier-league/startseite/wettbewerb/GB1/saison_id/",
    "https://www.transfermarkt.com/bundesliga/startseite/wettbewerb/L1/saison_id/",
    "https://www.transfermarkt.com/laliga/startseite/wettbewerb/ES1/saison_id/",
    "https://www.transfermarkt.com/serie-a/startseite/wettbewerb/IT1/saison_id/",
    "https://www.transfermarkt.com/ligue-1/startseite/wettbewerb/FR1/saison_id/"
]

def all_league_urls(url, season_range = [2010,2021]):
    league_url = []
    for i in url:
        league_url.append(list(map(lambda x : i + str(x), np.arange(season_range[0], season_range[1]+1, 1))))
    league_url  = list(itertools.chain(*league_url))
    return league_url
    

league_url = all_league_urls(leagues)
league_url

['https://www.transfermarkt.com/premier-league/startseite/wettbewerb/GB1/saison_id/2010',
 'https://www.transfermarkt.com/premier-league/startseite/wettbewerb/GB1/saison_id/2011',
 'https://www.transfermarkt.com/premier-league/startseite/wettbewerb/GB1/saison_id/2012',
 'https://www.transfermarkt.com/premier-league/startseite/wettbewerb/GB1/saison_id/2013',
 'https://www.transfermarkt.com/premier-league/startseite/wettbewerb/GB1/saison_id/2014',
 'https://www.transfermarkt.com/premier-league/startseite/wettbewerb/GB1/saison_id/2015',
 'https://www.transfermarkt.com/premier-league/startseite/wettbewerb/GB1/saison_id/2016',
 'https://www.transfermarkt.com/premier-league/startseite/wettbewerb/GB1/saison_id/2017',
 'https://www.transfermarkt.com/premier-league/startseite/wettbewerb/GB1/saison_id/2018',
 'https://www.transfermarkt.com/premier-league/startseite/wettbewerb/GB1/saison_id/2019',
 'https://www.transfermarkt.com/premier-league/startseite/wettbewerb/GB1/saison_id/2020',
 'https://

In [8]:
def find_team_urls(league_urls):
    # Teams
    headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.106 Safari/537.36'}
    team_url = []

    for i in league_urls:
        soup = BeautifulSoup(requests.get(i, headers=headers).content, "html.parser") 
        team_urls = soup.find("table", class_ = "items").find_all("a")
        team_url.append(pd.Series(list(map(lambda x: "https://www.transfermarkt.com" + x["href"], team_urls))).unique().tolist())
    
        # team_urls = soup.find("table", class_ = "items").find_all("a", {"class":"vereinprofil_tooltip"})
        
    team_url  = list(itertools.chain(*team_url))
    links = list(filter(lambda k: 'kader' in k, team_url))
    return links

team_url = find_team_urls(league_url)
print("Number of historical urls for teams:",len(team_url))
team_url[:5]


Number of historical urls for teams: 1176


['https://www.transfermarkt.com/chelsea-fc/kader/verein/631/saison_id/2010',
 'https://www.transfermarkt.com/manchester-united/kader/verein/985/saison_id/2010',
 'https://www.transfermarkt.com/manchester-city/kader/verein/281/saison_id/2010',
 'https://www.transfermarkt.com/liverpool-fc/kader/verein/31/saison_id/2010',
 'https://www.transfermarkt.com/arsenal-fc/kader/verein/11/saison_id/2010']

In [1]:
def find_player_urls(team_url):
    
    get_time("start")
    tm_player_url = []
    
    for completed,i in enumerate(team_url):
        headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.106 Safari/537.36'}
        soup = BeautifulSoup(requests.get(i, headers=headers).content, "html.parser") 
        p = soup.find("table", class_ = "items").find_all("a")
        tm_player_url.append(pd.Series(list(map(lambda x: "https://www.transfermarkt.com" + x["href"], p))).unique().tolist())

        # Print Message: How many teams have scraped?
        sys.stdout.write("\r{0} teams have just scraped from TM!".format(completed+1))
        sys.stdout.flush()

    get_time("end")
    tm_player_url  = pd.Series(list(itertools.chain(*tm_player_url))).unique().tolist()
    
    
    tm_player_url_df = pd.DataFrame({"TMURL":tm_player_url})
    tm_player_url_df["TMId"] = tm_player_url_df.TMURL.str.split("spieler/", expand = True)[1]

    # import re 
    # def Filter(string, substr): 
    #    return [str for str in string if
    #            any(sub in str for sub in substr)] 

    # get_time("start")
    # new_tm_player_url = []
    # for i in tm_player_url:
    #     soup = BeautifulSoup(requests.get(i, headers=headers).content, "html.parser") 
        
    #     # Kaleci is a Turkish word. It means Goal Keeper. It should change if there is a problem!
    #     if len(Filter(list(map(lambda x: x.text, soup.find_all("span", {"class":"dataValue"}))), ["Kaleci"])) < 1:
    #         new_tm_player_url.append(i)

    # get_time("end")

    # tm_player_url_df = pd.DataFrame({"TMURL":new_tm_player_url})
    # tm_player_url_df["TMId"] = tm_player_url_df.TMURL.str.split("spieler/", expand = True)[1]
    
    return tm_player_url_df.drop_duplicates()
    


# An example of Chelsea Players in 2017 season. Attention: I chose one team!
# tm_player_url_df = find_player_urls(team_url)
# print("Number of urls for players:",len(tm_player_url_df))
# tm_player_url_df

In [2]:
# tm_player_url_df.to_csv('transfermarkt_data.csv', index=False)

In [3]:
def market_value(url):
    
    # Url & Player Id
    url = url.replace("profil", "marktwertverlauf")
    playerid = url.split("spieler/")[1]
    
    # Request
    headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.106 Safari/537.36'}
    r = requests.get(url, headers=headers, timeout=1000)
    soup = BeautifulSoup(r.content, "html.parser")
    
    # Name
    if soup.find("h1") != None:
        name = soup.find("h1").get_text()
    
    # Parsed 
    script = soup.find("script", text=re.compile("Highcharts.Chart")).text
    parsed = pyjsparser.parse(script) 
    
    # Find Data from parsed info
    sc = parsed["body"][26]["expression"]["arguments"][1]["body"]["body"][1]["declarations"][0]["init"]["arguments"][0]["properties"][5]["value"]["elements"][0]["properties"][2]["value"]["elements"]
    mv = list(map(lambda x: int(x["properties"][0]["value"]["value"]), sc))
    club = list(map(lambda x: x["properties"][1]["value"]["value"], sc))
    date = list(map(lambda x: x["properties"][4]["value"]["value"], sc))
    
    mv_df = pd.DataFrame({"TMId":playerid,"Player":name,"MarketValue":mv, "Club":club, "Date":date})
    
    # Date format is Turkish. We have to arrange format. If you adapt all code by using English urls, you can remove this
    months = {"Oca":"January", "Şub":"February", "Mar":"March", "Nis":"April", "May":"May", "Haz":"June", 
              "Tem":"July", "Ağu":"August","Eyl":"September", "Eki":"October", "Kas":"November", "Ara":"December"}
    mv_df['Date']=pd.to_datetime(mv_df['Date'].replace(months, regex=True), dayfirst=True, errors = "coerce")
    mv_df = mv_df[mv_df.Date.notnull()]
    
    return mv_df

# Test
# mv = market_value("https://www.transfermarkt.com/mesut-ozil/marktwertverlauf/spieler/35664")
# mv

In [37]:

def injury_table(url):
    # URL & PLAYER ID
    url = url.replace("profil", "verletzungen")
    pid = url.split("spieler/")[1]

    # Request
    headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.106 Safari/537.36'}
    r=requests.get(url, headers = headers)
    soup=BeautifulSoup(r.content, "html.parser")
    
    if soup.find("h1") != None:
        name = soup.find("h1").get_text()
        nationality = soup.find("span", {"itemprop":"nationality"}).get_text()
        dateofbirth = soup.find("span", {"itemprop":"birthDate"}).get_text()
        height = soup.find("span", {"itemprop":"height"}).get_text()

    try:
        
        temp = pd.read_html(str(soup.find("table", class_ = "items")))[0]
        
        try:
            # Find page number
            page_numbers = []

            for i in soup.find("div", {'class' : "pager"}).find_all('li'):
                page_numbers.append(i.find('a')['title'])

            page =len(list(filter(lambda k: 'Page' in k, page_numbers)))
        
            if page > 1:
                for page_num in np.arange(2, page+1, 1):
                    url2 = url + "/ajax/yw1/page/"+str(page_num)
                    soup2 = BeautifulSoup(requests.get(url2, headers=headers).content, "html.parser")  
                    temp_table2 = pd.read_html(str(soup2.find("table", class_ = "items")))[0]
                    temp = temp.append(temp_table2)
            
        except:
            pass
        
        temp["TMId"]=pid
        temp['name']=name 
        temp['dateofbirth']=dateofbirth
        temp['nationality']=nationality
        temp['height']=height
        
        temp = temp.replace('\n', '', regex=True)
        
        return temp.reset_index(drop=True)
    
    except:
        pass
    
# Test
injury_table("https://www.transfermarkt.com/mesut-ozil/profil/spieler/35664")

,Season,Injury,from,until,Days,Games missed,TMId,name,dateofbirth,nationality,height
0,21/22,groin strain,"Oct 29, 2021","Nov 5, 2021",7 days,2,35664,Mesut Özil,"Oct 15, 19...",Germany,"1,80 m"
1,21/22,Influenza,"Sep 28, 2021","Oct 2, 2021",4 days,1,35664,Mesut Özil,"Oct 15, 19...",Germany,"1,80 m"
2,21/22,groin strain,"Aug 28, 2021","Sep 3, 2021",6 days,1,35664,Mesut Özil,"Oct 15, 19...",Germany,"1,80 m"
3,20/21,Unknown Injury,"May 11, 2021","May 16, 2021",5 days,2,35664,Mesut Özil,"Oct 15, 19...",Germany,"1,80 m"
4,20/21,Fitness,"Apr 9, 2021","Apr 26, 2021",17 days,3,35664,Mesut Özil,"Oct 15, 19...",Germany,"1,80 m"
5,20/21,Corona virus,"Mar 21, 2021","Apr 1, 2021",11 days,1,35664,Mesut Özil,"Oct 15, 19...",Germany,"1,80 m"
6,20/21,Ruptured ankle ligament,"Mar 5, 2021","Apr 9, 2021",35 days,5,35664,Mesut Özil,"Oct 15, 19...",Germany,"1,80 m"
7,20/21,Fitness,"Jan 25, 2021","Jan 31, 2021",6 days,2,35664,Mesut Özil,"Oct 15, 19...",Germany,"1,80 m"
8,20/21,Quarantine,"Jan 18, 2021","Jan 24, 2021",6 days,2,35664,Mesut Özil,"Oct 15, 19...",Germany,"1,80 m"
9,19/20,Back trouble,"Jun 27, 2020","Aug 2, 2020",36 days,10,35664,Mesut Özil,"Oct 15, 19...",Germany,"1,80 m"


In [5]:
player_urls = list(tm_player_url_df['TMURL'])

player_urls =list(filter(lambda k: 'profil' in k, player_urls))


player_injuries_df = pd.DataFrame(columns=['Season', 'Injury', 'from', 'until', 'Days', 'Games missed', 'TMId', 'name'])

for i in player_urls:
    df = injury_table(i)
    player_injuries_df = player_injuries_df.append(df)
    sys.stdout.write("\r{0} player injuries have just scraped from TM!".format(len(player_injuries_df)))
    sys.stdout.flush()
        
player_injuries_df.to_csv('player_injuries_df.csv', index=False)  

55216 player injuries have just scraped from TM!

In [6]:
player_injuries_df.head()

,Season,Injury,from,until,Days,Games missed,TMId,name
0,18/19,Hamstring Injury,"Sep 29, 2018","Oct 27, 2018",28 days,5,5658,Petr Cech
1,16/17,Calf Strain,"Mar 18, 2017","Apr 17, 2017",30 days,5,5658,Petr Cech
2,15/16,Calf Injury,"Mar 3, 2016","Mar 31, 2016",28 days,5,5658,Petr Cech
3,14/15,Neck Injury,"May 11, 2015","May 25, 2015",14 days,2,5658,Petr Cech
4,13/14,Shoulder Injury,"Apr 24, 2014","Jul 17, 2014",84 days,4,5658,Petr Cech


In [4]:
def profile_extract(url):
    # URL & PLAYER ID
    
    pid = url.split("spieler/")[1]

    job_lst = list(set(['Manager',
                'Assistant Manager',
                'Goalkeeping Coach',
                'Sporting Director',
                'Technical Coach',
                'Scout',
                'Team Manager',
                'Agent',
                'Technical Director',
                'Head of first te...',
                'Head of first team football',
                'Director of Foot...',
                'Director of Football',
                'Executive Assistant',
                'Player-Coach',
                'Academy Manager',
                'Individual Coach',
                'Director of yout...',
                'Director of youth department',
                'Member of the Su...',
                'Member of the Supervisory Board',
                'Director of Sport',
                'Academy manager',
                'Club representative',
                'Chief Executive ...',
                'Chief Executive Officer',
                'Squad leader',
                'Sports coordinator',
                'Youth Coach',
                'Director of Deve...',
                'Director of Development',
                'Sporting CEO',
                'Coach trainee',
                'Mental Coach',
                'Marketing Staff',
                'Academy Goalkeep...',
                'Academy Goalkeeping Co-ordinator',
                'Head of Soccer S...',
                'Coordinator firs...',
                'Coordinator of t...',
                'Coordinator of talent management',
                'Match Analyst',
                'Team coordinator',
                'Head of Scouting',
                'Advisor', 'President', 'Chief instructor', 'Caretaker Manager', 'Forward Coach', 'Loan Player Manager', 'Technical Assistant',
                'Team Manager', 'Vice-President', 'Chief Scout', 'Marketing/Manage...', 'Marketing/Management', 'Board Member', 'Director of Prof...', 
                'Director of Professional Football', 'Board member', 'Head of first te...', 'Head of first team football', 'Development Coach', 
                'Adivser of manag...', 'Secretary general', 'Staff of the office', 
                'Head of Internat...', 'Head of Football...', 'Team official', 'President of adm...', 'President of administrative board',
                'Strategic Advisor', 'Executive assist...', 'Executive assistant of the manager', 'Investor', '', 'Chairman',
                'Adivser of management', 'Head of International Relations and Scouting', 'Head of Football Operations', 'Head of first team football',
                'Coordinator of talent management', 'Manager',
                'Assistant Manager',
                'Goalkeeping Coach',
                'Sporting Director',
                'Scout',
                'Technical Coach',
                'Technical Director',
                'Team Manager',
                'Club representative',
                'Player-Coach',
                'Director of Football',
                'Owner',
                'Head of first team football',
                'Youth Coach',
                'Caretaker Manager',
                'Executive Assistant',
                'Director of Sport',
                'Team coordinator',
                'Coach trainee',
                'Sporting CEO',
                'Sports coordinator',
                'Loan Player Manager',
                'Advisor',
                'Chief Executive Officer',
                'Member of the Supervisory Board',
                'Director of youth department',
                'Forward Coach',
                'Academy manager',
                'Academy Goalkeeping Co-ordinator',
                'Academy Manager',
                'Board Member',
                'Marketing/Management',
                'Vice-President',
                'Individual Coach',
                'Match Analyst',
                'Coordinator of talent management',
                'Mental Coach',
                'Technical Assistant',
                'Squad leader',
                'Chief Scout',
                'President',
                'Umpire',
                'Chief instructor',
                'Executive assistant of the manager',
                'Head of Soccer School',
                'Adivser of management',
                'Staff of the office',
                'Secretary general',
                'Director of Development',
                'Head of Scouting',
                'Media worker',
                'President of administrative board',
                'Head of International Relations and Scouting',
                'Development Coach',
                'Marketing Staff',
                'Director of Professional Football',
                'Strategic Advisor',
                'Board member',
                'Team official',
                'Coordinator first team football',
                'Head of Football Operations',
                'Investor',
                'Marketing Staff']))
    
    # Request
    headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.106 Safari/537.36'}
    r=requests.get(url, headers = headers)
    soup=BeautifulSoup(r.content, "html.parser")

    column_names = []

    for i in soup.find_all("span", {'class':'dataItem'}):
        # if i.get_text() != 'club:':
        column_names.append(i.get_text())
        # else:
            # continue
    
    for i in soup.find_all("span", {'class':"info-table__content info-table__content--regular"}):
        column_names.append(i.get_text().strip())

    column_names = [s.strip() for s in column_names]

    column_values = []

    for i in soup.find_all("span", {'class':'dataValue'}):
        if i.get_text().strip() not in job_lst:
            column_values.append(i.get_text())
        else:
            continue
    
    for i in soup.find_all("span", {'class':"info-table__content info-table__content--bold"}):
        column_values.append(i.get_text().strip())

    column_values = [s.strip() for s in column_values]
    
    if soup.find("h1").get_text() != None:
        temp = pd.DataFrame(dict(zip(column_names, column_values)),index=[0])
        try:
            temp['name'] = soup.find("h1").get_text()
        except:
            pass
        try:
            temp["TMId"]=pid
        except:
            pass
        try:
            temp.drop(columns=['Date of birth/Age:'], axis=1, inplace=True)
        except:
            pass
        
    return temp
    
# Test

# profile_extract("https://www.transfermarkt.com/petr-cech/profil/spieler/5658")
# profile_extract("https://www.transfermarkt.com/lionel-messi/profil/spieler/28003")
# profile_extract('https://www.transfermarkt.com/claudio-pizarro/profil/spieler/532')
# profile_extract('https://www.transfermarkt.com/xavi/profil/spieler/7607')
# profile_extract('https://www.transfermarkt.com/zinedine-zidane/profil/spieler/3111')
# profile_extract('https://www.transfermarkt.com/javier-mascherano/profil/spieler/19981')
# profile_extract('https://www.transfermarkt.com/tim-cahill/profil/spieler/13556')
# profile_extract('https://www.transfermarkt.us/teofilo-cubillas/profil/spieler/133753')
# profile_extract('https://www.transfermarkt.us/juan-roman-riquelme/profil/spieler/3854')
# profile_extract('https://www.transfermarkt.com/iker-casillas/profil/spieler/3979')
# profile_extract('https://www.transfermarkt.us/oliver-kahn/profil/spieler/206')
# profile_extract('https://www.transfermarkt.us/gianluigi-buffon/profil/spieler/5023')
# profile_extract('https://www.transfermarkt.com/luis-figo/profil/spieler/3446')
# profile_extract('https://www.transfermarkt.us/adriano/profil/spieler/5876')

profile_extract('https://www.transfermarkt.com/roman-weidenfeller/profil/spieler/26')

,Place of birth:,Citizenship:,Height:,Position:,Former International:,Caps/Goals:,Last club:,Most games for:,Retired since:,club:,Date of birth:,Age:,Foot:,Player agent:,Current club:,Joined:,Contract expires:,Outfitter:,Social-Media:,name,TMId
0,Diez,Germany,"1,88 m",Goalkeeper,Germany,5/0,Bor. Dortmund,Bor. Dortmund,"Jul 1, 2018",Bor. Dortmund,"Aug 6, 1980",41,left,Jörg Neubauer,Retired,"Jul 1, 2018",-,Puma,,Roman Weidenfeller,26


In [4]:
tm_player_url_df = pd.read_csv('transfermarkt_data.csv')

In [9]:
tm_player_url_df = tm_player_url_df[~pd.isna(tm_player_url_df['TMId'])]
tm_player_url_df = tm_player_url_df.drop_duplicates()

tm_player_url_df_1 = tm_player_url_df[:4000]
tm_player_url_df_2 = tm_player_url_df[4000:8000]
tm_player_url_df_3 = tm_player_url_df[8000:]

len(tm_player_url_df), len(tm_player_url_df_1), len(tm_player_url_df_2), len(tm_player_url_df_3) 

(12902, 4000, 4000, 4902)

In [18]:
tm_player_url_df_1['TMURL'][6]

'https://www.transfermarkt.com/petr-cech/profil/spieler/5658'

In [10]:
# Sravan

player_urls = list(tm_player_url_df_1['TMURL'])

player_urls =list(filter(lambda k: 'profil' in k, player_urls))

player_profile_df = pd.DataFrame(columns=['Place of birth:', 'Citizenship:', 'Height:', 'Position:',
       'Contract expires:', 'Former International:', 'Caps/Goals:',
       'League level:', 'Joined:', 'Contract until:', 'name', 'TMId',
       'Date of birth', 'Last club:', 'Most games for:', 'Unknown since:',
       'Date of birth:', 'Age:', 'Foot:', 'Player agent:', 'Current club:',
       'Agent:', 'National player:', 'Name in home country:', 'Outfitter:',
       'Current international:', 'Social-Media:', 'Full name:',
       'On loan from:', 'Contract there expires:', 'Contract option:',
       'Retired since:', 'club:', 'Without Club since:', '2nd club:',
       'country:', 'Company:', 'Date of death:', 'Career break since:',
       '3nd club:', 'Ban since:'])



for i in player_urls:
    df = profile_extract(i)
    player_profile_df = player_profile_df.append(df)
    sys.stdout.write("\r{0} player injuries have just scraped from TM!".format(len(player_profile_df)))
    sys.stdout.flush()

4000 player injuries have just scraped from TM!

In [15]:
player_profile_df[player_profile_df['Contract expires:'] == 'Retired']

,Place of birth:,Citizenship:,Height:,Position:,Contract expires:,Former International:,Caps/Goals:,League level:,Joined:,Contract until:,name,TMId,Date of birth,Last club:,Most games for:,Unknown since:,Date of birth:,Age:,Foot:,Player agent:,Current club:,Agent:,National player:,Name in home country:,Outfitter:,Current international:,Social-Media:,Full name:,On loan from:,Contract there expires:,Contract option:,Retired since:,club:,Without Club since:,2nd club:,country:,Company:,Date of death:,Career break since:,3nd club:,Ban since:
0,Marketing Staff,41,Diez,"1,88 m",Retired,Germany,5/0,NaN,Jörg Neubauer,NaN,Roman Weidenfeller,26,NaN,Bor. Dortmund,Bor. Dortmund,NaN,Bor. Dortmund,"Aug 6, 1980",Germany,Goalkeeper,left,NaN,NaN,NaN,"Jul 1, 2018",NaN,-,NaN,NaN,NaN,NaN,"Jul 1, 2018",Marketing Staff,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
player_profile_df.to_csv('player_profile_df_1.csv', index=False)  

In [13]:
# Renzo 

player_urls = list(tm_player_url_df_2['TMURL'])

player_urls =list(filter(lambda k: 'profil' in k, player_urls))

player_profile_df = pd.DataFrame(columns=['Place of birth:', 'Citizenship:', 'Height:', 'Position:',
                                           'Contract expires:', 'Former International:', 'Caps/Goals:',
                                           'League level:', 'Joined:', 'Contract until:', 'name', 'TMId',
                                           'Date of birth'])

for i in player_urls:
    df = profile_extract(i)
    player_profile_df = player_profile_df.append(df)
    sys.stdout.write("\r{0} player injuries have just scraped from TM!".format(len(player_profile_df)))
    sys.stdout.flush()
        
  

4000 player injuries have just scraped from TM!

NameError: name 'player_injuries_df' is not defined

In [14]:
player_profile_df.to_csv('player_profile_df_2.csv', index=False)

In [ ]:
# Victor 

player_urls = list(tm_player_url_df_3['TMURL'])

player_urls =list(filter(lambda k: 'profil' in k, player_urls))

player_profile_df = pd.DataFrame(columns=['Place of birth:', 'Citizenship:', 'Height:', 'Position:',
                                           'Contract expires:', 'Former International:', 'Caps/Goals:',
                                           'League level:', 'Joined:', 'Contract until:', 'name', 'TMId',
                                           'Date of birth'])

for i in player_urls:
    df = profile_extract(i)
    player_profile_df = player_profile_df.append(df)
    sys.stdout.write("\r{0} player injuries have just scraped from TM!".format(len(player_profile_df)))
    sys.stdout.flush()
        
player_injuries_df.to_csv('player_profile_df_3.csv', index=False)  